### 셋업

In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/al02524278/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/al02524278/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/al02524278/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 텍스트 전처리

In [14]:
text = "Nvidia's Huang says nuclear power an option to feed data warehouse"

In [15]:
tokens = text.split()
tokens

["Nvidia's",
 'Huang',
 'says',
 'nuclear',
 'power',
 'an',
 'option',
 'to',
 'feed',
 'data',
 'warehouse']

In [16]:
# NLTK 이용 토큰화
from nltk.tokenize import word_tokenize
word_tokens = word_tokenize(text)
word_tokens

['Nvidia',
 "'s",
 'Huang',
 'says',
 'nuclear',
 'power',
 'an',
 'option',
 'to',
 'feed',
 'data',
 'warehouse']

In [17]:
# 불용어 필터링
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_words = [w for w in word_tokens if not w.lower() in stop_words]
filtered_words

['Nvidia',
 "'s",
 'Huang',
 'says',
 'nuclear',
 'power',
 'option',
 'feed',
 'data',
 'warehouse']

In [18]:
# 어간 추출
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in filtered_words]
stemmed_words

['nvidia',
 "'s",
 'huang',
 'say',
 'nuclear',
 'power',
 'option',
 'feed',
 'data',
 'warehous']

In [19]:
# 기본형 추출
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
lemmatized_words

['Nvidia',
 "'s",
 'Huang',
 'say',
 'nuclear',
 'power',
 'option',
 'feed',
 'data',
 'warehouse']

### Batch Index Pipeline

### 셋업

In [20]:
# Skip if you already have Java

# # Install Homebrew
# /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# # Install OpenJDK
# brew install openjdk@17

# # set JAVA_HOME in zshrc
# export JAVA_HOME=$(/usr/libexec/java_home -v17)
# export PATH=$JAVA_HOME/bin:$PATH

In [21]:
# Skip if you already have Spark

# # Download and install Spark
# ! curl -O https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
# ! tar -xzf spark-3.4.3-bin-hadoop3.tgz
# # Move to /opt folder
# ! sudo mv spark-3.4.3-bin-hadoop3 /opt/spark-3.4.3
# cp /opt/spark-3.4.3/conf/spark-defaults.conf.template /opt/spark-3.4.3/conf/spark-defaults.conf
# echo 'spark.driver.extraJavaOptions   -Djava.security.manager=allow' >> /opt/spark-3.4.3/conf/spark-defaults.conf
# echo 'spark.executor.extraJavaOptions   -Djava.security.manager=allow' >> /opt/spark-3.4.3/conf/spark-defaults.conf
# # Create symlink
# ! sudo ln -s /opt/spark-3.4.3 /opt/spark

# # set SPARK_HOME in .zshrc
# export SPARK_HOME=/opt/spark
# export PATH=$SPARK_HOME/bin:$PATH

# # Jupyter Notebook Spark integration
# export PYSPARK_PYTHON=$(brew --prefix python)/libexec/bin/python
# export PYSPARK_DRIVER_PYTHON='jupyter'
# export PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port=8889'
# # Download elastic-spark jar from https://central.sonatype.com/artifact/org.elasticsearch/elasticsearch-spark-30_2.12/versions
# export PYSPARK_SUBMIT_ARGS='--jars /opt/spark-3.4.3/jars/elasticsearch-spark-30_2.12-8.15.0.jar pyspark-shell'

In [22]:
# Skip if you already have Elasticsearch

# Download Docker from https://docs.docker.com/engine/install/

# # pull docker images
# docker pull docker.elastic.co/elasticsearch/elasticsearch:8.15.0
# docker pull docker.elastic.co/kibana/kibana:8.15.0

# # start Elasticsearch and Kibana containers
# docker network create elastic
# docker run -d --name elasticsearch --net elastic -p 9200:9200 -p 9300:9300 -m 1GB -e "discovery.type=single-node" -e "ELASTIC_PASSWORD=password" -e "xpack.security.enabled=false" -e "xpack.security.enrollment.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.15.0
# docker run -d --name kibana --net elastic -p 5601:5601 docker.elastic.co/kibana/kibana:8.15.0

In [1]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf()
spark = SparkSession.builder.appName("indexer").config(conf=conf).getOrCreate()

# Dataset: https://www.kaggle.com/datasets/ahsenwaheed/youtube-comments-spam-dataset?resource=download
path = "Youtube-Spam-Dataset.csv"

df = spark.read.csv(path)
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/09 23:01:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/09 23:01:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|                 _c0|                 _c1|                _c2|                 _c3|                 _c4|  _c5|
+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|          COMMENT_ID|              AUTHOR|               DATE|             CONTENT|          VIDEO_NAME|CLASS|
|LZQPQhLyRh80UYxNu...|           Julius NM|2013-11-07T06:20:48|Huh, anyway check...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh_C2cTtd...|         adam riyati|2013-11-07T12:37:15|Hey guys check ou...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh9MSZYnf...|    Evgeny Murashkin|2013-11-08T17:34:21|just for test I h...|PSY - GANGNAM STY...|    1|
|z13jhp0bxqncu512g...|     ElNino Melendez|2013-11-09T08:28:43|me shaking my sex...|PSY - GANGNAM STY...|    1|
|z13fwbwp1oujthgqj...|              GsMega|2013-11-10T16:05:38|watch?v=vtaRGgvGt...|PSY - GANGNAM STY...

In [2]:
columns = df.columns
new_df = df.filter(df._c0 != 'COMMENT_ID')
new_df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+---+
|                 _c0|                 _c1|                _c2|                 _c3|                 _c4|_c5|
+--------------------+--------------------+-------------------+--------------------+--------------------+---+
|LZQPQhLyRh80UYxNu...|           Julius NM|2013-11-07T06:20:48|Huh, anyway check...|PSY - GANGNAM STY...|  1|
|LZQPQhLyRh_C2cTtd...|         adam riyati|2013-11-07T12:37:15|Hey guys check ou...|PSY - GANGNAM STY...|  1|
|LZQPQhLyRh9MSZYnf...|    Evgeny Murashkin|2013-11-08T17:34:21|just for test I h...|PSY - GANGNAM STY...|  1|
|z13jhp0bxqncu512g...|     ElNino Melendez|2013-11-09T08:28:43|me shaking my sex...|PSY - GANGNAM STY...|  1|
|z13fwbwp1oujthgqj...|              GsMega|2013-11-10T16:05:38|watch?v=vtaRGgvGt...|PSY - GANGNAM STY...|  1|
|LZQPQhLyRh9-wNRtl...|        Jason Haddad|2013-11-26T02:55:11|Hey, check out my...|PSY - GANGNAM STY...|  1|
|z13lfzdo5

In [3]:
df_schema = df.schema

new_columns = []
for i in df.limit(1).collect()[0]:
  new_columns.append(i)

for i,k in enumerate(df_schema.fields):
  k.name = new_columns[i]

# Apply the schema to the original dataframe
new_df = spark.createDataFrame(new_df.rdd, df_schema)
new_df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|          COMMENT_ID|              AUTHOR|               DATE|             CONTENT|          VIDEO_NAME|CLASS|
+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
|LZQPQhLyRh80UYxNu...|           Julius NM|2013-11-07T06:20:48|Huh, anyway check...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh_C2cTtd...|         adam riyati|2013-11-07T12:37:15|Hey guys check ou...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh9MSZYnf...|    Evgeny Murashkin|2013-11-08T17:34:21|just for test I h...|PSY - GANGNAM STY...|    1|
|z13jhp0bxqncu512g...|     ElNino Melendez|2013-11-09T08:28:43|me shaking my sex...|PSY - GANGNAM STY...|    1|
|z13fwbwp1oujthgqj...|              GsMega|2013-11-10T16:05:38|watch?v=vtaRGgvGt...|PSY - GANGNAM STY...|    1|
|LZQPQhLyRh9-wNRtl...|        Jason Haddad|2013-11-26T02:55:11|Hey, check out my...|PSY - GANGNAM STY...

In [4]:
# Write the DataFrame to an Elasticsearch index
es_conf = {
    "es.nodes.discovery": "false",
    "es.nodes.data.only": "false",
    "es.net.http.auth.user": "elastic",
    "es.net.http.auth.pass": "password",
    "es.index.auto.create": "true",
    "es.nodes": "http://127.0.0.1",
    "es.port": "9200",
    "es.mapping.id": "COMMENT_ID",
}

new_df.write.mode("append") \
        .format('org.elasticsearch.spark.sql') \
        .options(**es_conf) \
        .save("youtube")

24/10/09 23:02:25 ERROR NetworkClient: Node [172.24.0.4:9200] failed (org.elasticsearch.hadoop.thirdparty.apache.commons.httpclient.ConnectTimeoutException: The host did not accept the connection within timeout of 60000 ms); selected next node [127.0.0.1:9200]
